In [1]:
from typing import List
import numpy as np

In [2]:
# toy example: you left the house and realise you don't have your keys. where are they?
# this is the most naive an inefficient text adventure on purpose.

#world = [('description', reward, [nested_world])]
class find_the_keys():
    "A little test world to get going"
    def __init__(self):
        self.world = (
            'look around', 0, [
                ('go left', 0, [
                    ('climb tree', 0, []),
                    ('search floor', 0, [
                        ('lift stone', 0,[]),
                        ('lift leaf', 0,[])
                    ])
                ]),
                ('go straight', 0, [
                    ('enter house', 0, [
                        ('check cupboard', 1, []),
                        ('check wardrobe', 0, [])
                    ])
                ]),
                ('go right', 0, [
                    ('check bike', 0, []), # bike is locked
                    ('check mailbox', 0, [
                        ('open first letter', 0, []),
                        ('open second letter', 0, []),
                        ('open third letter', 0, []),
                    ])
                ])
            ]
        )
        self.reset()
        
        
    def _get_state(self):
        state = self.world
        description = state[0]
        reward = state[1]
        actions = state[2]
        for a in self.previous_actions:
            description, reward, actions = actions[a]
        return description, reward, actions
    
    def _set_state(self, previous_actions):
        self.previous_actions = previous_actions
        self.description, self.reward, self.actions = self._get_state()
        
    def reset(self):
        self._set_state([])
    
    def get_description(self):
        return self.description
    
    def get_reward(self):
        return self.reward
    
    def get_actions(self):
        return [(a[0], a[1]) for a in self.actions] + [('go back', 0)]
    
    def do(self, action):
        if action == 'go back':
            self._set_state(self.previous_actions[:-1])
            return "done"
        else:
            for i, a in enumerate(game.get_actions()):
                if a[0] == action:
                    self._set_state(self.previous_actions + [i])
                    return "done"
        return "impossible"

In [3]:
game = find_the_keys()

In [4]:
game.get_description()

'look around'

In [5]:
game.get_reward()

0

In [6]:
game.get_actions()

[('go left', 0), ('go straight', 0), ('go right', 0), ('go back', 0)]

In [7]:
game.do('go left')

'done'

In [8]:
game.get_actions()

[('climb tree', 0), ('search floor', 0), ('go back', 0)]

In [9]:
game.do('go back')

'done'

In [10]:
game.get_actions()

[('go left', 0), ('go straight', 0), ('go right', 0), ('go back', 0)]

In [11]:
# how to solve
game.do('go straight')
game.do('enter house')
game.do('check cupboard')
print(game.reward)

1


In [12]:
# random exploration
game = find_the_keys()
steps = 9000
rewards = []
rng = np.random.default_rng()
for i in range(steps):
    description = game.get_description()
    actions = game.get_actions()
    rewards.append(game.get_reward())
    #print(description, reward)
    if rewards[-1]:
        # this only makes sense because of the specific game here
        #print("You won! Reseting game.")
        game._set_state([])
    else:
        j = rng.integers(0, len(actions))
        #print("do", actions[j][0])
        game.do(actions[j][0])

In [13]:
sum(rewards)

105

In [30]:
# q-learning
# again the most naive way I could think of, see https://en.wikipedia.org/wiki/Q-learning
game = find_the_keys()
rewards = []

class q_learner():
    """Naive q-learning agent
    
    Implementing reward table "q[state,action]" as a key-value map
    """
    def __init__(
        self,
        q_init:float = 1, # initial q > 0 incentivises exploration
        learning_rate:float = .9, # "alpha"
        discount_factor:float = .1, # "gamma"
        p_explore:float = 0.01,
        rng:np.random.Generator = np.random.default_rng() # allows to control randomness
    ):
        self.q = {} # the reward "table"
        self.q_init = q_init
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.p_explore = p_explore
        self.rng = rng
        self.verbose = False
    
    def get_q_s(self,state:str, actions:list):
        q_s = self.q.get(state, {})
        q_s = {a[0]: self.q_init for a in actions} | q_s # fill q for missing actions
        return q_s
    
    def choose_action(self, q_s, state, actions):
        "explore or choose action maximising q"
        #q_s = self.get_q_s(state, actions)
        if self.p_explore > self.rng.uniform():
            if self.verbose: print("explore")
            j = self.rng.integers(0, len(actions))
            action = actions[j][0]
        else:
            if self.verbose: print("exploit")
            if self.verbose: print(q_s)
            action = max(q_s, key=q_s.get)
        return action
    
    def learn(self, q_s, action, new_state, new_actions):
        max_new_q = max(agent.get_q_s(new_state, new_actions).values())
        q_s = self.get_q_s(state, actions)
        q_sa = q_s.get(action, self.q_init)
        if verbose: print("q_sa", q_sa)
        q_s[action] = (
            (1 - self.learning_rate) * q_sa
            + self.learning_rate * (new_reward + self.discount_factor * max_new_q)
        )
        self.q[state] = q_s
        if verbose: print(f"new q", self.q)


agent = q_learner()
state = game.get_description()
actions = game.get_actions()
steps = 9000
verbose=False
agent.verbose=verbose

for i in range(steps):
    if verbose: print()
    if verbose: print("step", i, "state", state)
    
    q_s = agent.get_q_s(state, actions)
    action = agent.choose_action(q_s, state, actions)
    
    if verbose: print("do", action)
    game.do(action)

    new_state = game.get_description()
    new_reward = game.get_reward()
    new_actions = game.get_actions()
    
    if verbose: print("new resward", new_reward, "max new q", max_new_q)

    agent.learn(q_s, action, new_state, new_actions)
    if verbose: print("q", agent.q)
    
    rewards.append(new_reward)
    
    if new_reward:
        game.reset()
        new_state = game.get_description()
        new_actions = game.get_actions()

    state = new_state
    actions = new_actions  

In [31]:
actions

[('check cupboard', 1), ('check wardrobe', 0), ('go back', 0)]

In [32]:
sum(rewards)

2932

In [33]:
agent.q

{'look around': {'go left': 0.00011000063279800003,
  'go straight': 0.011000000000000003,
  'go right': 0.00011000000076868903,
  'go back': 0.0011057590000000002},
 'go left': {'climb tree': 0.00024760989999999996,
  'search floor': 0.0004247568999999999,
  'go back': 0.0011000000575900003},
 'climb tree': {'go back': 0.0002345041},
 'search floor': {'lift stone': 0.00024760989999999996,
  'lift leaf': 0.0004444398999999999,
  'go back': 0.000281152},
 'lift stone': {'go back': 0.00024760989999999996},
 'lift leaf': {'go back': 0.0003205098999999999},
 'go straight': {'enter house': 0.11000000000000001,
  'go back': 0.0011000000000116789},
 'enter house': {'check cupboard': 1.1,
  'check wardrobe': 0.011000659600000004,
  'go back': 0.011000098900000003},
 'go right': {'check bike': 0.00024760989999999996,
  'check mailbox': 0.00024760989999999996,
  'go back': 0.0011000000000575903},
 'check bike': {'go back': 0.00024760989999999996},
 'check mailbox': {'open first letter': 0.000247